In [1]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\ITSC\Desktop\project\DL\data\nsmc\ratings.txt", sep = "\t")

In [3]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [4]:
df.shape

(200000, 3)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [6]:
# 결측치 제거
df = df.dropna()

In [7]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [8]:
df.shape

(199992, 3)

In [9]:
komoran = Komoran()

In [10]:
%%time
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
docs = df["document"].map(lambda x: komoran.nouns(x))

CPU times: total: 1min 55s
Wall time: 1min 24s


In [11]:
docs

0                                                       [때]
1         [디자인, 학생, 외국, 디자이너, 전통, 발전, 문화, 산업, 사실, 우리나라, ...
2                                    [폴리스, 스토리, 시리즈, 뉴, 최고]
3                                [연기, 것, 라고, 생각, 몰입, 영, 화지]
4                                        [안개, 밤하늘, 초승달, 영화]
                                ...                        
199995                                         [포켓, 몬스터, 짜]
199996                                                   []
199997                            [완전, 사이코, 영화, 마지막, 영화, 질]
199998                                          [라따뚜이, 스머프]
199999                       [포, 풍, 저그, 가나, 신다영, 차영, 차영, 차]
Name: document, Length: 199992, dtype: object

In [12]:
df.iloc[-1]

id                  8548411
document    포풍저그가나가신다영차영차영차
label                     0
Name: 199999, dtype: object

In [13]:
# word2vec 모델 학습
model = Word2Vec(sentences = docs, vector_size = 200, window = 4, min_count = 2, sg = 1)

- word2vec의 주요 하이퍼파라미터
    - sentences : Word2Vec 모델 학습에 필요한 문장 데이터
    - vector_size : 단어 임베딩 벡터의 차원(크기)
    - window : 주변 단어 윈도우의 크기
    - min_count : 단어 최소 빈도 수 제한(설정된 min_count 빈도 수 미만의 단어들은 학습하지 않음)
    - sg : 0(CBOW 모델), 1(skip-gram  모델)

In [14]:
# 모델 저장
model.save("./model/nsmc_w2v.model")

In [15]:
# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count :", model.corpus_count)
print("corpus_total_words :", model.corpus_total_words)

corpus_count : 199992
corpus_total_words : 1076840


# word2vec 모델 활용

In [16]:
# 모델 로딩
model = Word2Vec.load("./model/nsmc_w2v.model")
print(model.corpus_total_words)

1076840


In [17]:
# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print("사랑 : ", model.wv["사랑"])

사랑 :  [-0.07630999 -0.404828    0.03590605  0.26142055 -0.10799312 -0.20799005
  0.15578352 -0.06889922 -0.04667915  0.37246004 -0.31771198  0.08706474
  0.34264183 -0.24531986 -0.08500683  0.44058338  0.13099106 -0.08209526
 -0.46247247 -0.37306425 -0.00394232  0.2229737   0.02022794  0.058669
 -0.23450816 -0.14988768 -0.06896751 -0.3427782  -0.18499257 -0.33115122
  0.04690713  0.25386924  0.0124919   0.19192344 -0.06699996  0.07476515
 -0.04299118  0.11017019 -0.24343836 -0.02289758 -0.3186641  -0.05258316
  0.01413407 -0.12597069  0.5164969   0.37820083 -0.34130797 -0.17770645
  0.34784573  0.13818377  0.3320351  -0.27600017 -0.20718794  0.1562726
 -0.20736124 -0.06994574  0.07881343 -0.27587542 -0.26469508 -0.02843448
  0.22658446  0.20734683 -0.17579609  0.16372548 -0.39259905  0.06278718
 -0.22637601  0.18521637 -0.25880623  0.912969    0.0870223  -0.03673316
  0.07412452  0.11765447  0.23368743 -0.40211624  0.14199342 -0.44768366
 -0.47046325 -0.19383918 -0.14639255  0.04659194

- 모델을 학습할 때 설정한 vector_size 하이퍼파라미터 만큼 단어 임베딩 벡터 차원 크기가 결정됨

- model.wv.similarity() 함수 : 두 단어 간의 유사도를 계산

In [20]:
# 단어 유사도 계산
print("일요일 = 월요일", model.wv.similarity(w1 = "일요일", w2 = "월요일"), sep = "\t")

일요일 = 월요일	0.90013766


In [21]:
print("대기업 = 삼성", model.wv.similarity(w1 = "대기업", w2 = "삼성"), sep = "\t")

대기업 = 삼성	0.87478346


In [22]:
print("일요일 = 삼성", model.wv.similarity(w1 = "일요일", w2 = "삼성"), sep = "\t")

일요일 = 삼성	0.6372349


- model.wv.similarity() 함수 : 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환
    - 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환
    - topn : 반환되는 유사한 단어 수
- 유사도가 1에 가까울수록 두 단어는 동일한 의미이거나 문법적으로 관련이 있을 가능성이 높음

In [23]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("시리즈", topn = 5))

[('엑스맨', 0.8122462034225464), ('포터', 0.8070046901702881), ('반지의 제왕', 0.8044701218605042), ('스타워즈', 0.7970862984657288), ('미이라', 0.7911685109138489)]


In [25]:
print(model.wv.most_similar("킹", topn = 5))

[('라이온', 0.870712161064148), ('명곡', 0.8090930581092834), ('출', 0.8024001121520996), ('스카이', 0.7918542623519897), ('결정판', 0.7915890216827393)]


In [26]:
import gensim

In [27]:
sentences = gensim.models.word2vec.Text8Corpus("./data/text8.txt")

In [30]:
model = Word2Vec(sentences, vector_size = 200)

In [31]:
model.wv.most_similar(positive = ["woman", "king"], negative = ["man"])

[('queen', 0.6435863971710205),
 ('princess', 0.5680335760116577),
 ('throne', 0.5655045509338379),
 ('prince', 0.5455148816108704),
 ('empress', 0.53355473279953),
 ('daughter', 0.5310457348823547),
 ('isabella', 0.5277004837989807),
 ('emperor', 0.5272979736328125),
 ('consort', 0.5272188186645508),
 ('wife', 0.5254906415939331)]